In [1]:
import requests
import pandas as pd 
import json
import concurrent.futures
import re
import queue
import os
from bs4 import BeautifulSoup
import lxml

from collections import defaultdict
import unicodedata

In [3]:
_name = pd.read_csv("CHĂM SÓC GIA ĐÌNH.csv")
_name.columns

Index(['tên sách', 'ảnh bìa', 'thể loại', 'tác giả', 'nội dung tóm tắt',
       'giá bìa', 'nhà phát hành', 'nhà xuất bản', 'ngày xuất bản',
       'kích thước', 'loại bìa', 'số trang', 'dịch giả'],
      dtype='object')

In [7]:
df2 = pd.read_csv("DATA_v2.csv")
df2.dropna(inplace = True) 
df2.columns = ["Thể loại", "Nguồn nhập", "type"]

# Remove all row that not have fahasa
df2 = df2[df2['Nguồn nhập'].str.contains("vina")]
group_f1 = df2.groupby("Thể loại")["Nguồn nhập"].apply(list).to_dict()
group_f2 = df2.groupby("type")["Thể loại"].apply(list).to_dict()

n_dict = defaultdict(dict)
for k, v in group_f2.items():
    for stype in v:
        if (len(group_f1[stype])) == 1:
            n_dict[k][stype] = group_f1[stype][0] + 'page-{}/?sef_rewrite=1'
        else:
            for _ in group_f1[stype]:
                n_dict[k][stype] = _ + 'page-{}/?sef_rewrite=1'

In [43]:
n_dict

defaultdict(dict,
            {'CHĂM SÓC GIA ĐÌNH': {'Hạnh phúc gia đình': 'https://www.vinabook.com/c513/hanh-phuc-gia-dinh/page-{}/?sef_rewrite=1'},
             'HỌC NGÔN NGỮ': {'Tiếng Trung': 'https://www.vinabook.com/c412/tieng-hoa/page-{}/?sef_rewrite=1',
              'Tiếng Pháp ': 'https://www.vinabook.com/c411/tieng-phap/page-{}/?sef_rewrite=1',
              'Tiếng Nhật ': 'https://www.vinabook.com/c413/tieng-nhat-1/page-{}/?sef_rewrite=1'},
             'NGÀNH CÔNG NGHỆ - THÔNG TIN': {'Cơ sở dữ liệu': 'https://www.vinabook.com/c399/co-so-du-lieu/page-{}/?sef_rewrite=1',
              'Hệ điều hành': 'https://www.vinabook.com/c400/he-dieu-hanh/page-{}/?sef_rewrite=1',
              'Internet - mạng': 'https://www.vinabook.com/c398/internet-mang/page-{}/?sef_rewrite=1',
              'Lập trình - Phần mềm ứng dụng': 'https://www.vinabook.com/c397/lap-trinh-phan-mem-ung-dung/page-{}/?sef_rewrite=1',
              'Lập trình web': ' https://www.vinabook.com/c396/lap-trinh-web/p

In [8]:
url = "https://www.vinabook.com/c495/co-vua-co-tuong/page-2/?sef_rewrite=1"

payload={}
headers = {
  'authority': 'www.vinabook.com',
  'pragma': 'no-cache',
  'cache-control': 'no-cache',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
  'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  'sec-fetch-site': 'none',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-user': '?1',
  'sec-fetch-dest': 'document',
  'sec-ch-ua': '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'accept-language': 'vi,en-US;q=0.9,en;q=0.8,vi-VN;q=0.7',
}

response = requests.request("GET", url, headers=headers, data=payload)

In [9]:
_big_dict = {}
booksQueue = queue.Queue()

class Crawler():
    def __init__(self, url, cname):
        self.url = url
        self.category_name = cname
        
    def crawl(self):
        _concurrent = self._bookshelf()
        if _concurrent != None:
            with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()*3) as executor:
                executor.map(self._books, _concurrent)
            
    def _books(self, _url,):
        
        book = {}
        __res =  requests.request("GET",_url, headers=headers)
        _soup = BeautifulSoup(__res.text, "html.parser")
        
        book['tên sách'] = _soup.h1.text
        book['ảnh bìa'] = _soup.find("div",{"class":"cm-image-wrap"}).img['src']
        book['thể loại'] = self.category_name
        
        _specific = _soup.find("div",{"class":"product-feature"})
        for _ in _specific.findAll("li"):
            _regex = re.sub('\s+',' ',_.text).split(":")
            book[_regex[0].lower()] = _regex[1]
            
        book['nội dung tóm tắt'] = _soup.find("div",{"class":"full-description"}).text.split("...")[0]
        book['giá bìa'] = _soup.find("span",{"class":"list-price nowrap"}).text
        
        print(book)
        booksQueue.put(book)
        print(f"Number of products in my pocket {booksQueue.qsize()}")
        
    def _bookshelf(self):
        
        _bookshelf = []
        print(self.url.format(48, 1))
        _res = requests.request("GET", self.url.format(1), headers=headers)
        soup = BeautifulSoup(_res.text, "html.parser")
        
        if soup.find("span",{"class":"group-paging-label"}):
            _pages = int(soup.find("span",{"class":"group-paging-label"}).text.split("/")[1]) + 1

            for _ in range(_pages):
                _ =+1
                _res = requests.request("GET", self.url.format(_), headers=headers)
                soup = BeautifulSoup(_res.text, "html.parser")
                for _ in soup.findAll("p",{"class":"price-info-nd"}):
                    _bookshelf.append(_.a['href'].rstrip())

            return _bookshelf
        else:
            return None

In [10]:
t = Crawler('https://www.vinabook.com/c398/internet-mang/page-{}/?sef_rewrite=1','Internet')
# print(t._bookshelf())
# print(t._books('https://www.vinabook.com/tam-quoc-dien-nghia-bo-3-cuon--p93877.html'))
t.crawl()

https://www.vinabook.com/c398/internet-mang/page-48/?sef_rewrite=1
{'tên sách': 'Khám phá Internet Mỗi Ngày', 'ảnh bìa': 'https://www.vinabook.com/images/thumbnails/product/240x/14150_p20543.jpg', 'thể loại': 'Internet', ' tác giả': ' Việt Văn Books ', ' nhà xuất bản': ' Nxb Thống kê ', ' nhà phát hành': ' Huy Hoàng ', ' mã sản phẩm': ' 8935073017513 ', ' khối lượng': ' 420.00 gam ', ' kích thước': ' 16x24 cm ', ' ngày phát hành': ' 12/2006 ', ' số trang': ' 344 ', 'nội dung tóm tắt': '  Thông tin tác giả    Việt Văn Books \n                                                                                                                                            Việt Văn Books\n                                                                   Vào trang riêng của tác giả   Xem tất cả các sách của tác giả    \n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tNgày nay công nghệ thông tin nói riêng và Internet nói chung, đã mang đến cho con người\xa0 về thông tin liên lạc ngày một hoàn thiện h

{'tên sách': 'Triển Khai Hệ Thống Mạng Wireless', 'ảnh bìa': 'https://www.vinabook.com/images/thumbnails/product/240x/44519_51062.jpg', 'thể loại': 'Internet', ' tác giả': ' Tô Thanh Hải ', ' nhà xuất bản': ' Nxb Lao động - Xã hội ', ' nhà phát hành': ' Minh khai ', ' mã sản phẩm': ' 8935087501558 ', ' khối lượng': ' 220.00 gam ', ' định dạng': ' Bìa mềm ', ' kích thước': ' 16 x 24 cm ', ' ngày phát hành': ' 03/2010 ', ' số trang': ' 172 ', 'nội dung tóm tắt': '  Thông tin tác giả    Tô Thanh Hải     Vào trang riêng của tác giả   Xem tất cả các sách của tác giả     Triển Khai Hệ Thống Mạng WirelessMạng Wireless đang ngày càng trở nên phổ biến. Với những ưu điểm như giá cả hợp lý, tính di động cao và cấu hình đơn giản, mạng wireless đã và đang trở thành một lựa chọn tất yếu để triển khai ở gia đình, doanh nghiệp và những nơi công cộng.Đối với phạm vi gia đình (bao gồm cả những doanh nghiệp nhỏ, nơi mà hạ tầng công nghệ thông tin chưa đáp ứng được yêu cầu triển khai các dịch vụ máy chủ),

Number of products in my pocket 12{'tên sách': 'Kết Nối Mạng Vô Tuyến Nhanh Chóng Và Hiệu Quả', 'ảnh bìa': 'https://www.vinabook.com/images/thumbnails/product/240x/294791_p80328mp32481m34c121be84c8f2aktnimngvtuynnhanhchngvhiuqu.jpg', 'thể loại': 'Internet', ' tác giả': ' VN-Guide ', ' nhà xuất bản': ' Nxb Thống kê ', ' nhà phát hành': ' Văn Lang ', ' mã sản phẩm': ' 1130000080328 ', ' khối lượng': ' 260.00 gam ', ' định dạng': ' Bìa mềm ', ' kích thước': ' 13.5x21 cm ', ' ngày phát hành': ' 09/2008 ', ' số trang': ' 264 ', 'nội dung tóm tắt': '  Thông tin tác giả    VN-Guide \n                                                                                                                                            VN-Guide\n                                                                   Vào trang riêng của tác giả   Xem tất cả các sách của tác giả     Kết nối mạng vô tuyến - nhanh chóng và hiệu quả hé mở những bí quyết độc đáo, cung cấp nhiều thủ thuật giúp tiết kiệm thời gian, cùng

{'tên sách': 'Kiến Trúc Máy Tính Tiên Tiến', 'ảnh bìa': 'https://www.vinabook.com/images/thumbnails/product/240x/289020_p79481m49788.jpg', 'thể loại': 'Internet', ' tác giả': ' TS. Trần Công Hùng ', ' nhà xuất bản': ' Nxb Thông tin & truyền thông ', ' nhà phát hành': ' NXB Thông Tin và Truyền Thông ', ' mã sản phẩm': ' 1130000079481 ', ' khối lượng': ' 462.00 gam ', ' định dạng': ' Bìa mềm ', ' kích thước': ' 19 x 27 cm ', ' ngày phát hành': ' 07/2011 ', ' số trang': ' 264 ', 'nội dung tóm tắt': '  Thông tin tác giả    TS. Trần Công Hùng     Vào trang riêng của tác giả   Xem tất cả các sách của tác giả     Máy tính còn gọi là máy vi tính hay điện toán, là những thiết bị hay hệ thống dùng để tính toán hay kiểm soát các hoạt động mà có thể biểu diễn dưới dạng số hay quy luật logic.Trong quyển sách này, chúng ta sẽ nghiên cứu và khảo sát về tổ chức, hoạt động của máy tính gọi là tiên tiến vào thời điểm hiện nay. Nội dung cuốn sách gồm 6 chương:Chương 1: Giới thiệu chungChương 2: Khối xử l

{'tên sách': 'Quản Trị Mạng - Window Server 2008 Dành Cho Người Tự Học - Tập 1', 'ảnh bìa': 'https://www.vinabook.com/images/thumbnails/product/240x/42379_48922.jpg', 'thể loại': 'Internet', ' tác giả': ' Ngọc Bích , Tường Thụy ', ' nhà xuất bản': ' Nxb Thông tin & truyền thông ', ' nhà phát hành': ' Thuận tâm huy ', ' khối lượng': ' 352.00 gam ', ' định dạng': ' Bìa mềm ', ' kích thước': ' 16 x 24 cm ', ' ngày phát hành': ' 02/2012 ', ' số trang': ' 288 ', 'nội dung tóm tắt': '  Thông tin tác giả    Ngọc Bích \n                                                                                                                                            Ngọc Bích\n                                                                   Vào trang riêng của tác giả   Xem tất cả các sách của tác giả       Tường Thụy     Vào trang riêng của tác giả   Xem tất cả các sách của tác giả     Sách biên soạn dành cho những người sắp trở thành nhà quản trị mạng cũng như đang làm công tác quản trị mạng. Sách 

...

{'tên sách': 'Hacker Lược Sử', 'ảnh bìa': 'https://www.vinabook.com/images/thumbnails/product/240x/326311_p86019mhacker.jpg', 'thể loại': 'Internet', ' tác giả': ' Steven Levy ', ' người dịch': ' Phan Anh Vũ ', ' nhà xuất bản': ' Nxb Công Thương ', ' nhà phát hành': ' Alpha books ', ' mã sản phẩm': ' 8935251411768 ', ' khối lượng': ' 990.00 gam ', ' ngôn ngữ': ' Tiếng Việt ', ' định dạng': ' Bìa mềm ', ' kích thước': ' 16 x 24 cm ', ' ngày phát hành': ' 12/2018 ', ' số trang': ' 640 ', 'nội dung tóm tắt': '  Thông tin tác giả    Steven Levy     Vào trang riêng của tác giả   Xem tất cả các sách của tác giả     Cuốn sách nói về những nhân vật, cỗ máy, sự kiện định hình cho văn hóa và đạo đức hacker từ những hacker đời đầu ở đại học MIT.Câu chuyện hấp dẫn bắt đầu từ các phòng thí nghiệm nghiên cứu máy tính đầu tiên đến các máy tính gia đình.Tập hợp tài liệu cập nhật từ các tin tặc nổi tiếng như Bill Gates, Mark Zuckerberg, Richard Stallman…Những sự thật về cuộc sống và con đường trở thành

{'tên sách': 'Nhập Môn Mạng Máy Tính', 'ảnh bìa': 'https://www.vinabook.com/images/thumbnails/product/240x/49200_55743.jpg', 'thể loại': 'Internet', ' tác giả': ' John Ross ', ' người dịch': ' FPT Polytechnic ', ' nhà xuất bản': ' NXB Tri thức ', ' nhà phát hành': ' Alpha books ', ' mã sản phẩm': ' 9786049083563 ', ' khối lượng': ' 506.00 gam ', ' định dạng': ' Bìa mềm ', ' kích thước': ' 19 x 27 cm ', ' ngày phát hành': ' 03/2013 ', ' số trang': ' 259 ', 'nội dung tóm tắt': '  Thông tin tác giả    John Ross     Vào trang riêng của tác giả   Xem tất cả các sách của tác giả     Nhập Môn Mạng Máy Tính - Hướng dẫn tổ chức và sử dụng mạng máy tínhPhải chăng các máy móc trong văn phòng của bạn đang hoạt động đơn độc? Bạn có một vài máy tính ở nhà và muốn kết nối với Internet? Cách tốt nhất để chia sẻ file trong một nhóm máy tính là tạo lập một mạng. Vậy bạn thực hiện điều đó bằng cách nào?Nhập Môn Mạng Máy Tính cung cấp nh

{'tên sách': 'Hướng Dẫn Sử Dụng Internet Cho Người Mới Bắt Đầu', 'ảnh bìa': 'https://www.vinabook.com/images/thumbnails/product/240x/44547_51090.jpg', 'thể loại': 'Internet', ' tác giả': ' Minh Tuấn , Hoàng Nguyên ', ' nhà xuất bản': ' NXB Hồng Đức ', ' nhà phát hành': ' 45 Đinh Tiên Hoàng ', ' mã sản phẩm': ' 113000051090 ', ' khối lượng': ' 484.00 gam ', ' định dạng': ' Bìa mềm ', ' kích thước': ' 16 x 24 cm ', ' ngày phát hành': ' 09/2012 ', ' số trang': ' 372 ', 'nội dung tóm tắt': '  Thông tin tác giả    Minh Tuấn     Vào trang riêng của tác giả   Xem tất cả các sách của tác giả       Hoàng Nguyên \n                                                                                                                                            Hoàng Nguyên\n                                                                   Vào trang riêng của tác giả   Xem tất cả các sách của tác giả     Hướng Dẫn Sử Dụng Internet Cho Người Mới Bắt ĐầuĐây là một cuốn sách đề cập đến lãnh vực sử dụng và k

{'tên sách': 'Người Chơi Facebook Khôn Ngoan Biết Rằng...', 'ảnh bìa': 'https://www.vinabook.com/images/thumbnails/product/240x/211212_p57474m003.jpg', 'thể loại': 'Internet', ' tác giả': ' Xuân Nguyễn ', ' nhà xuất bản': ' Nxb Trẻ ', ' nhà phát hành': ' NXB Trẻ ', ' mã sản phẩm': ' 8934974126669 ', ' khối lượng': ' 264.00 gam ', ' ngôn ngữ': ' Tiếng Việt ', ' định dạng': ' Bìa mềm ', ' kích thước': ' 13 x 20 cm ', ' ngày phát hành': ' 03/2016 ', ' số trang': ' 276 ', 'nội dung tóm tắt': '  Thông tin tác giả    Xuân Nguyễn \n                                                                                                                                            Xuân Nguyễn\n                                                                   Vào trang riêng của tác giả   Xem tất cả các sách của tác giả     Dù muốn dù không, mạng xã hội đã trở thành một phần quan trọng trong cuộc sống của con người. Trong số đó, Facebook đã nổi lên thành ông trùm của thời đại sống ảo – một thời đại mà

{'tên sách': 'Công Nghệ Mạng Máy Tính - Tái bản 09/10/2010', 'ảnh bìa': 'https://www.vinabook.com/images/thumbnails/product/240x/34926_41469.jpg', 'thể loại': 'Internet', ' tác giả': ' TS. Phạm Thế Quế ', ' nhà xuất bản': ' Nxb Thông tin & truyền thông ', ' nhà phát hành': ' Quang Minh ', ' khối lượng': ' 520.00 gam ', ' định dạng': ' Bìa mềm ', ' kích thước': ' 14.5 x 20.5 cm ', ' ngày phát hành': ' 09/2010 ', ' số trang': ' 528 ', 'nội dung tóm tắt': '  Thông tin tác giả    TS. Phạm Thế Quế     Vào trang riêng của tác giả   Xem tất cả các sách của tác giả     Cuốn sách gồm các nội dung chính sau:Chương 1: Khái niệm cơ bản về mạng máy tính, bao gồm các nội dung về mục tiêu và lợi ích khi kết nối mạng máy tính, khái niệm cấu trúc mạng và giao thức mạng, mạng số liệu, môi trường truyền dẫn và một số vấn đề cơ bản khi thiết kế kiến trúc mạng, đánh giá độ tin cậy của mạng.Chương 2: Mô hình các hệ thống mở OSI, nghiên cứu các nguyên tắc cơ bản để thiết kế một mô hình giao thức mạng máy tín

In [38]:
for _name in n_dict.keys():
    booksQueue = queue.Queue()
    for k,v in n_dict[_name].items():
        print(v)
        t = Crawler(v, k)
        t.crawl()

    total += booksQueue.qsize()
    df = pd.DataFrame(columns=__name.columns,index=[0])
    while not booksQueue.empty():
        df = df.append(booksQueue.get(),ignore_index=True)

    df.to_csv('vina/fahasa_'+ _name+'.csv',index=False)

https://www.vinabook.com/c513/hanh-phuc-gia-dinh/page-{}/?sef_rewrite=1
https://www.vinabook.com/c412/tieng-hoa/page-{}/?sef_rewrite=1
https://www.vinabook.com/c411/tieng-phap/page-{}/?sef_rewrite=1
https://www.vinabook.com/c413/tieng-nhat-1/page-{}/?sef_rewrite=1
https://www.vinabook.com/c399/co-so-du-lieu/page-{}/?sef_rewrite=1
https://www.vinabook.com/c400/he-dieu-hanh/page-{}/?sef_rewrite=1
https://www.vinabook.com/c398/internet-mang/page-{}/?sef_rewrite=1
https://www.vinabook.com/c397/lap-trinh-phan-mem-ung-dung/page-{}/?sef_rewrite=1
 https://www.vinabook.com/c396/lap-trinh-web/page-{}/?sef_rewrite=1
https://www.vinabook.com/c401/phan-cung/page-{}/?sef_rewrite=1
https://www.vinabook.com/c394/tin-hoc-co-ban/page-{}/?sef_rewrite=1
https://www.vinabook.com/c395/tin-hoc-van-phong/page-{}/?sef_rewrite=1
https://www.vinabook.com/c402/do-hoa/page-{}/?sef_rewrite=1
https://www.vinabook.com/c403/sach-tin-hoc-bang-ngoai-ngu/page-{}/?sef_rewrite=1
https://www.vinabook.com/c447/cau-duong/pag